In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["font.size"] = 12

## install dependencies

In [ ]:
# !mamba install -c conda-forge -y proj-data sentinelsat xmlschema

In [ ]:
# !pip install elevation sarsen xarray-sentinel

## processing definition

In [ ]:
# dem_urlpath = "South-of-Redmond-10m-small.tif"
dem_urlpath = "South-of-Redmond-10m.tif"

# product_folder = "GRD/2021/12/19/IW/DV/S1B_IW_GRDH_1SDV_20211219T020132_20211219T020157_030088_0397B7_70CF"
product_folder = "GRD/2021/12/17/IW/DV/S1B_IW_GRDH_1SDV_20211217T141304_20211217T141329_030066_039705_9048"
measurement_group = "IW/VV"

## download DEM

In [ ]:
import os

import adlfs
import numpy as np
import planetary_computer
import pystac_client
import rioxarray  # enable the `.rio` accessor
import stackstac
import xarray as xr

from sarsen import apps, scene

In [ ]:
seattle = [-121.96, 47.05]
areas_of_interest = {"type": "Point", "coordinates": seattle}

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
threedep = catalog.get_child(id="3dep-seamless")

longitude, latitude = seattle
buffer = 0.05
bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]
search = catalog.search(collections="3dep-seamless", bbox=bbox)
items = list(search.get_items())
items

In [ ]:
items_high_res = [
    planetary_computer.sign(item).to_dict()
    for item in items
    if item.properties["gsd"] == 10
]

dem_raster_all = stackstac.stack(items_high_res, bounds=bbox).squeeze()
dem_raster_all

In [ ]:
# dem_raster = dem_raster_all.compute().mean("time")
# dem_raster.rio.set_crs(dem_raster_all.rio.crs)
# dem_raster.rio.to_raster(dem_urlpath)

In [ ]:
dem_raster = scene.open_dem_raster(dem_urlpath)
_ = dem_raster.plot()

## download data

In [ ]:
import os


def mirror_folder(fs, bucket, folder, exclude="vh"):
    for path, folders, files in fs.walk(f"{bucket}/{folder}"):
        os.makedirs(path[len(bucket) + 1 :], exist_ok=True)
        for f in files:
            if exclude in f:
                continue
            file_path = os.path.join(path, f)
            lfile_path = file_path[len(bucket) + 1 :]
            if not os.path.isfile(lfile_path):
                print(file_path)
                fs.download(file_path, lfile_path + "~")
                os.rename(lfile_path + "~", lfile_path)

In [ ]:
grd_account_name = "sentinel1euwest"
grd_bucket = "s1-grd"
grd_token = planetary_computer.sas.get_token(grd_account_name, grd_bucket).token

grd_product_folder = f"{grd_bucket}/{product_folder}"

grd_fs = adlfs.AzureBlobFileSystem(grd_account_name, credential=grd_token)
grd_fs.ls(grd_product_folder)

In [ ]:
# mirror_folder(grd_fs, grd_bucket, product_folder)

In [ ]:
ds = xr.open_dataset(
    product_folder,
    engine="sentinel-1",
    group=measurement_group,
    # storage_options=storage_options,
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
)
ds

In [ ]:
rtc_account_name = "sentinel1euwestrtc"
rtc_bucket = "sentinel1-grd-rtc"
rtc_token = planetary_computer.sas.get_token(rtc_account_name, rtc_bucket).token

rtc_product_folder = f"{rtc_bucket}/{product_folder}"

rtc_fs = adlfs.AzureBlobFileSystem(rtc_account_name, credential=rtc_token)
rtc_fs.ls(rtc_product_folder)

In [ ]:
# mirror_folder(rtc_fs, rtc_bucket, product_folder)

## process

In [ ]:
%%time
gtc_path = os.path.basename(product_folder) + ".GTC.tif"

apps.backward_geocode_sentinel1(
    product_folder,
    measurement_group,
    dem_urlpath,
    interp_method="nearest",
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
    output_urlpath=gtc_path,
)

In [ ]:
gtc = xr.open_dataarray(gtc_path).drop("band")
gtc

In [ ]:
gtc.plot(vmax=0.5)

In [ ]:
%%time
rtc_cosine_path = os.path.basename(product_folder) + ".RTC.cosine.tif"

apps.backward_geocode_sentinel1(
    product_folder,
    measurement_group,
    dem_urlpath,
    interp_method="nearest",
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
    correct_radiometry="cosine",
    output_urlpath=rtc_cosine_path,
)

In [ ]:
rtc_cosine = xr.open_dataarray(rtc_cosine_path).drop("band")
rtc_cosine

In [ ]:
rtc_cosine.plot(vmax=0.2)

In [ ]:
%%time
rtc_gamma_path = os.path.basename(product_folder) + ".RTC.gamma.tif"

apps.backward_geocode_sentinel1(
    product_folder,
    measurement_group,
    dem_urlpath,
    interp_method="nearest",
    override_product_files="{dirname}/{prefix}{swath}-{polarization}{ext}",
    chunks={"slant_range_time": 2048},
    correct_radiometry="gamma",
    output_urlpath=rtc_gamma_path,
)

In [ ]:
rtc_gamma = xr.open_dataarray(rtc_gamma_path).drop("band")
rtc_gamma

In [ ]:
rtc_gamma.plot(vmax=0.5)